# Querying the data

In [0]:
# Run after mount_s3_load_data with clean_data appended 

In [0]:
from pyspark.sql.window import Window

df_pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)
# create join of df_pin and df_user and a temporary view to run SQL query to create age group column
df_pin.join(df_user, df_pin.ind == df_user.ind).createOrReplaceTempView("category_age")
age_groups = spark.sql(
    "SELECT CASE \
        WHEN age between 18 and 24 then '18-24' \
        WHEN age between 25 and 35 then '25-35' \
        WHEN age between 36 and 50 then '36-50' \
        WHEN age > 50 then '50+' \
        END as age_group, * FROM category_age")

###Find the most popular category in each country

In [0]:
# order by category_count column and partition by country column
window_popular_category = Window.partitionBy("country").orderBy(col("category_count").desc())
# use aggregation function with df_pin_geo dataframe and window function to find the most popular category in each country
df_pin_geo.groupBy("country", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("ranking", row_number().over(window_popular_category)) \
.filter(col("ranking") == 1).drop("ranking") \
.show()

In [0]:
%sql
/*
+--------------------+--------------+--------------+
|             country|      category|category_count|
+--------------------+--------------+--------------+
|         Afghanistan|     education|             3|
|             Albania|        beauty|             5|
|             Algeria|        quotes|             7|
|      American Samoa|       tattoos|             2|
|             Andorra|       tattoos|             3|
|              Angola|diy-and-crafts|             2|
|            Anguilla|diy-and-crafts|             2|
|Antarctica (the t...|       tattoos|             2|
| Antigua and Barbuda|     christmas|             5|
|           Argentina|       tattoos|             2|
|             Armenia|diy-and-crafts|             2|
|               Aruba|  mens-fashion|             3|
|           Australia|  mens-fashion|             2|
|             Austria|        travel|             2|
|          Azerbaijan|event-planning|             1|
|             Bahamas|event-planning|             1|
|             Bahrain|       finance|             1|
|          Bangladesh|    home-decor|             2|
|            Barbados|     education|             2|
|             Belgium|        travel|             1|
+--------------------+--------------+--------------+
only showing top 20 rows
*/ 


### Find which was the most popular category each year

In [0]:
# order by category_count column and partition by post_year column
window_popular_category_year = Window.partitionBy("post_year").orderBy(col("category_count").desc())
# use aggregation function with df_pin_geo dataframe and window function to find the most popular category in each year
df_pin_geo.withColumn("post_year", year("timestamp")) \
.filter(col("post_year") >= 2018).filter(col("post_year") <= 2022) \
.groupBy("post_year", "category").agg(count("category").alias("category_count")) \
.withColumn("ranking", row_number().over(window_popular_category_year)) \
.filter(col("ranking") == 1).drop("ranking") \
.show()

In [0]:
%sql
/*
+---------+--------------+--------------+
|post_year|      category|category_count|
+---------+--------------+--------------+
|     2018|     education|             9|
|     2019|diy-and-crafts|            10|
|     2020|        travel|             8|
|     2021|        quotes|             8|
|     2022|        beauty|            11|
+---------+--------------+--------------+
*/

### Find the user with the most followers in each country

In [0]:
# order by follower_count column and partition by country column
window_followers_by_country = Window.partitionBy("country").orderBy(col("follower_count").desc())
# use df_pin_geo dataframe and window function to find the user with the most followers in each country
max_followers = df_pin_geo.withColumn("ranking", row_number().over(window_followers_by_country)) \
    .filter(col("ranking") == 1).select("country", "poster_name", "follower_count")

max_followers.show()

In [0]:
%sql
/*
+--------------------+--------------------+--------------+
|             country|         poster_name|follower_count|
+--------------------+--------------------+--------------+
|         Afghanistan|                9GAG|       3000000|
|             Albania|   The Minds Journal|       5000000|
|             Algeria|           YourTango|        942000|
|      American Samoa|         Mamas Uncut|       8000000|
|             Andorra|Teachers Pay Teac...|       1000000|
|              Angola|           Tastemade|       8000000|
|            Anguilla|Kristen | Lifesty...|         92000|
|Antarctica (the t...|          Refinery29|       1000000|
| Antigua and Barbuda|Country Living Ma...|       1000000|
|           Argentina|         Next Luxury|        800000|
|             Armenia|Michelle {CraftyM...|        892000|
|               Aruba|         GQ Magazine|        874000|
|           Australia|   Cultura Colectiva|       1000000|
|             Austria|The World Pursuit...|         89000|
|          Azerbaijan|     Style Me Pretty|       6000000|
|             Bahamas|Her Packing List ...|         41000|
|             Bahrain|R.J. Weiss at The...|         46000|
|          Bangladesh|Better Homes and ...|       4000000|
|            Barbados|The Creativity Ex...|        410000|
|             Belgium|Bon Traveler | Tr...|         24000|
+--------------------+--------------------+--------------+
only showing top 20 rows
*/

### Find the country with the user with most followers

In [0]:
# find max followers from max_followers dataframe
max_user_followers = max_followers.select(max("follower_count")).collect()[0][0]
# use top result of max_followers dataframe to find the country with the user with the most followers
country_max_followers = max_followers.select("*").where(col("follower_count") == max_user_followers)
country_max_followers.show()

In [0]:
%sql
/*
+--------------+-----------+--------------+
|       country|poster_name|follower_count|
+--------------+-----------+--------------+
|American Samoa|Mamas Uncut|       8000000|
|        Angola|  Tastemade|       8000000|
+--------------+-----------+--------------+
*/

### Find the most popular category for different age groups

In [0]:
# order by category_count column and partition by age_group column
window_popular_category_by_age = Window.partitionBy("age_group").orderBy(col("category_count").desc())
# from age_groups use aggregation function to find the most popular category by age group
age_groups.groupBy("age_group", "category").agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(window_popular_category_by_age)) \
.filter(col("rank") == 1).drop("rank") \
.show()

In [0]:
%sql
/*
+---------+--------+--------------+
|age_group|category|category_count|
+---------+--------+--------------+
|    18-24| tattoos|            16|
|    25-35| finance|            11|
|    36-50|  quotes|             8|
|      50+|  beauty|             4|
+---------+--------+--------------+
*/

### Find the median follower count for different age groups

In [0]:
# from age_groups use aggregation function to find the median followers ordered by age group
age_groups.select("user_name", "date_joined", "age_group", "follower_count") \
.distinct().groupBy("age_group").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("age_group") \
.show()

In [0]:
%sql
/*
+---------+---------------------+
|age_group|median_follower_count|
+---------+---------------------+
|    18-24|                59000|
|    25-35|                31000|
|    36-50|                 6000|
|      50+|                 3000|
+---------+---------------------+
*/

### Find how many users joined each year

In [0]:
# user user_df and aggregation function to find how many users joined in each year
df_user.withColumn("post_year", year("date_joined")).drop("ind").distinct() \
.filter(col("post_year") >= 2015).filter(col("post_year") <= 2020) \
.groupBy("post_year").agg(count("user_name").alias("number_users_joined")) \
.orderBy("post_year") \
.show()

In [0]:
%sql
/*
+---------+-------------------+
|post_year|number_users_joined|
+---------+-------------------+
|     2015|                 85|
|     2016|                136|
|     2017|                 49|
+---------+-------------------+
*/

### Find the median follower count of users based on their joining year

In [0]:
# use age_groups and aggregation function to find the median followers based on their joining year
age_groups.select("user_name", "date_joined", "follower_count") \
.distinct().withColumn("post_year", year("date_joined")) \
.groupBy("post_year").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year") \
.show()

In [0]:
%sql
/*
+---------+---------------------+
|post_year|median_follower_count|
+---------+---------------------+
|     2015|                59000|
|     2016|                22000|
|     2017|                 6000|
+---------+---------------------+
*/

### Find the median follower count of users based on their joining year and age group

In [0]:
# use age_groups and aggregation function to find the median followers of users based on their joining year and age group
age_groups.select("user_name", "age_group", "date_joined", "follower_count") \
.distinct().withColumn("post_year", year("date_joined")) \
.groupBy("post_year", "age_group").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("post_year", "age_group") \
.show()

In [0]:
%sql
/*
+---------+---------+---------------------+
|post_year|age_group|median_follower_count|
+---------+---------+---------------------+
|     2015|    18-24|               211000|
|     2015|    25-35|                42000|
|     2015|    36-50|                13000|
|     2015|      50+|                14000|
|     2016|    18-24|                40000|
|     2016|    25-35|                27000|
|     2016|    36-50|                 9000|
|     2016|      50+|                 1000|
|     2017|    18-24|                11000|
|     2017|    25-35|                 8000|
|     2017|    36-50|                 3000|
|     2017|      50+|                 5000|
+---------+---------+---------------------+
*/